<h2><b>VGG19 Model with Batch Normalization</b></h2><br>
<h4><b>Import Libraries</b></h4>

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

<h4><b>Genarating DataTransformer

In [13]:
transform = transforms.Compose([    transforms.Resize((32, 32)), 
                                    transforms.ToTensor(), 
                                    transforms.Normalize((0.5,), (0.5,))
                                ])

<h4><b>Download the MNIST Dataset

In [14]:
train_dataset = torchvision.datasets.MNIST(root='./dataset', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./dataset', train=False, transform=transform, download=True)

<h4><b>Split the Data into Training set and Validation set

In [15]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

<h4><b>Build the DataLoader

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

<h4><b>Build the VGG19 Model with Batch Normalization

In [ ]:
model = torchvision.models.vgg19_bn(pretrained=True)

# 修改第一層卷積層的輸入通道數
model.features[0] = nn.Conv2d(3, 64, kernel_size=3, padding=1)  

# 修改分類器部分，以符合MNIST的類別數（10類）
model.classifier[-1] = nn.Linear(4096, 10)

# 將模型轉移到GPU上（如果可用的話）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

<h4><b>Records Parameters

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

<h4><b>Training Process

In [ ]:
epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # 在驗證集上評估模型
    model.eval()
    correct_val = 0
    total_val = 0
    running_val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

<h4><b>Evaluating Model on Testing Dataset

In [ ]:
# 在測試集上評估模型
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

<h4><b>Saving the Model

In [ ]:
torch.save(model, './VGG19_BN_model.pth')

<h4><b>Saving the Figure

In [ ]:
# 保存訓練/驗證損失和準確度圖

# 創建第一張圖（第一個子圖）
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('training_loss_validation_loss.png')
plt.show()

# 創建第二張圖（第二個子圖）
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig('training_accuracy_validation_accuracy.png')
plt.show()